In [54]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import Embedding

In [112]:
df = pd.read_csv('./IMDB Dataset.csv')

In [113]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [114]:
# Remove html tags in review column with regex
df = df.replace(to_replace ='<[^>]*>', value = '', regex = True)

In [115]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [116]:
df.shape

(50000, 2)

In [105]:
# Convert seentiment to numeric. In this case, 0 for positive and 1 for negative
sentiment_label = df.sentiment.factorize()
sentiment_label

(array([0, 0, 1, ..., 1, 1, 1]),
 Index(['neutral', 'negative', 'positive'], dtype='object'))

In [106]:
# Assign a number to each word and replace with that number in the sentence

# Get the actual texts from the data frame
review = df.review.values
# Initialize the tokenizer with a 5000 word limit. This is the number of words we would like to encode
tokenizer = Tokenizer(num_words=5000)
# Call fit_on_texts to create associations of words and numbers as shown in the image below.
tokenizer.fit_on_texts(review)

vocab_size = len(tokenizer.word_index) + 1
encoded_docs = tokenizer.texts_to_sequences(review)
# As the model input requires to have equal length, the sequence is padded to have a length
padded_sequence = pad_sequences(encoded_docs, maxlen=200)

In [107]:
# Review and the corresponding numeric representation of it
print(review[0][:200])
print(padded_sequence[0])

According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0

In [108]:
# Build the model

embedding_vector_length = 32

model = Sequential()

model.add(Embedding(vocab_size, embedding_vector_length, input_length=len(padded_sequence[0])))
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 200, 32)           323680    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 200, 32)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                16600     
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 340,331
Trainable params: 340,331
Non-trainable params: 0
_________________________________________________________________
None


In [109]:
history = model.fit(padded_sequence,sentiment_label[0],
                  validation_split=0.2, epochs=5, batch_size=32)

Epoch 1/5
122/122 [==============================] - 26s 195ms/step - loss: 0.5888 - accuracy: 0.1024 - val_loss: 0.4524 - val_accuracy: 0.6216
Epoch 2/5
122/122 [==============================] - 22s 180ms/step - loss: -0.9957 - accuracy: 0.3782 - val_loss: 0.7070 - val_accuracy: 0.6608
Epoch 3/5
122/122 [==============================] - 24s 198ms/step - loss: -2.5762 - accuracy: 0.4462 - val_loss: 0.2814 - val_accuracy: 0.6629
Epoch 4/5
122/122 [==============================] - 23s 187ms/step - loss: -4.9363 - accuracy: 0.4983 - val_loss: 0.4185 - val_accuracy: 0.6742
Epoch 5/5
122/122 [==============================] - 22s 179ms/step - loss: -6.3439 - accuracy: 0.5329 - val_loss: 0.2611 - val_accuracy: 0.6588


In [75]:
def analyze_sentence(sentence):
    tw = tokenizer.texts_to_sequences([sentence])
    tw = pad_sequences(tw, maxlen=200)
    prediction = int(model.predict(tw).round().item())
    print(model.predict(tw))
    return sentiment_label[1][prediction]

In [81]:
print(analyze_sentence("I saw this when it first came out and was shocked even then by the complete failure of the Hollywood hacks who made the film to either put Conan on the screen or even just make a decent movie. Phew. Recently, I tried to watch it again, twenty-odd years later - and I gagged. It's irredeemable. It's on a par with Plan 9. Only it's more boring."))

[[0.9490154]]
negative


In [117]:
model.save('./imdb_sentiment_analyzer')

INFO:tensorflow:Assets written to: ./imdb_sentiment_analyzer/assets
